In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Adding Necessary Imports

In [30]:
import pandas as pd
import numpy as np
import re
import string
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tabulate import tabulate


Loading Data --> Renaming Columns  --> Merging content data with labels data

In [31]:
def load_data():
    """Load and preprocess the datasets."""
    content_df = pd.read_csv("/content/drive/MyDrive/Home_depo/content_data_MASTER.csv")  # Contains 'title', 'slug'
    labels_df = pd.read_csv("/content/drive/MyDrive/Home_depo/labels_MASTER.csv")      # Has incorrect column names
    test_df = pd.read_csv("/content/drive/MyDrive/Home_depo/test_MASTER.csv")          # Contains 'searchTerm'

    # Rename columns in labels_df
    labels_df.rename(columns={'#4 #14 connector': 'searchTerm',
                              'types-of-pipe-fittings': 'slug',
                              'RELEVANT': 'label'}, inplace=True)

    # Merge labels with content to get titles (only labeled rows will have a title)
    labels_df = labels_df.merge(content_df, on="slug", how="left")

    # Remove rows with missing titles, drop duplicates, and reset index
    labels_df.dropna(subset=["title"], inplace=True)
    labels_df.drop_duplicates(inplace=True)
    labels_df.reset_index(drop=True, inplace=True)
    return content_df, labels_df, test_df


Preprocess text by lowercasing, removing punctuation, and extra spaces.

In [32]:
def preprocess_text(text):
    """Preprocess text by lowercasing, removing punctuation, and extra spaces."""
    text = str(text).lower()
    text = re.sub(f"[{string.punctuation}]", "", text)  # Remove punctuation
    text = re.sub(r"\s+", " ", text).strip()             # Remove extra spaces
    return text

Apply text preprocessing to search terms and titles.

In [33]:
def preprocess_data(labels_df):
    """Apply text preprocessing to search terms and titles."""
    labels_df["processed_search"] = labels_df["searchTerm"].apply(preprocess_text)
    labels_df["processed_title"] = labels_df["title"].apply(preprocess_text)
    labels_df["processed_slug"] = labels_df["slug"].apply(preprocess_text)
    return labels_df

Vectorize processed text using Hugging Face Sentense Transformer all-MiniLM-L6-v2

In [34]:
def fit_sentence_embeddings(labels_df):
    """Generate sentence embeddings."""
    model = SentenceTransformer("all-MiniLM-L6-v2")
    sentence_embeddings = model.encode(labels_df["processed_title"].tolist(), convert_to_numpy=True)
    return model, sentence_embeddings

Compute Cosine similarity scores using Sentence Tranformer vectors

In [35]:
def compute_similarity(search_term, model, sentence_embeddings):
    """Compute similarity scores using sentence embeddings."""
    search_term_processed = preprocess_text(search_term)

    # Compute similarity with Sentence Embeddings
    search_embedding = model.encode([search_term_processed], convert_to_numpy=True)
    embedding_similarities = cosine_similarity(search_embedding, sentence_embeddings).flatten()
    return embedding_similarities


Find the best threshold for classifying search terms as RELEVANT or NOT RELEVANT.

In [36]:
def find_best_threshold(labels_df, model, sentence_embeddings):
    """Find the best threshold for classifying search terms as RELEVANT or NOT RELEVANT."""
    thresholds = np.linspace(0, 1, 101)
    best_threshold = None
    best_f1 = 0

    # Split labeled data for evaluation
    X_train, X_test, y_train, y_test = train_test_split(labels_df["processed_search"], labels_df["label"], test_size=0.2, random_state=42)

    # Mapping for binary evaluation
    label_mapping = {"NOT RELEVANT": 0, "RELEVANT": 1}
    y_test_num = [label_mapping[label] for label in y_test]

    for t in thresholds:
        preds = []
        for search_term in X_test:
            scores = compute_similarity(search_term, model, sentence_embeddings)
            pred = "RELEVANT" if max(scores) > t else "NOT RELEVANT"
            preds.append(pred)

        y_pred_num = [label_mapping[p] for p in preds]
        f1 = f1_score(y_test_num, y_pred_num, average="macro")
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = t

    return best_threshold, best_f1


**Model Evaluation**

In [37]:
def evaluate_model(labels_df, model, sentence_embeddings, threshold):
    """Evaluate model performance using the best threshold."""
    X_train, X_test, y_train, y_test = train_test_split(labels_df["processed_search"], labels_df["label"], test_size=0.2, random_state=42)
    predictions = []

    for search_term in X_test:
        scores = compute_similarity(search_term, model, sentence_embeddings)
        predicted_label = "RELEVANT" if max(scores) > threshold else "NOT RELEVANT"
        predictions.append(predicted_label)

    print(classification_report(y_test, predictions))

Generating Recommendations for the test set that we have been given

In [38]:
def recommend_articles_for_test(test_df, labels_df, model, sentence_embeddings, threshold, top_k=5):
    """Generate recommendations for the test set, removing duplicates."""
    recommendations = {}

    for search_term in test_df["searchTerm"]:
        scores = compute_similarity(search_term, model, sentence_embeddings)
        top_indices = np.argsort(scores)[::-1][:top_k]
        top_scores = scores[top_indices]

        if max(top_scores) < threshold:
            recommendations[search_term] = "Result not found"
        else:
            # Get candidate rows: each row is a list [slug, title, processed_title]
            candidate_rows = labels_df.iloc[top_indices][["slug", "title", "processed_title"]].values.tolist()

            # Remove duplicates by slug
            seen_slugs = set()
            unique_candidates = []

            for candidate in candidate_rows:
                slug, title, _ = candidate

                if slug not in seen_slugs:
                    unique_candidates.append([slug, title])
                    seen_slugs.add(slug)

            recommendations[search_term] = unique_candidates

    return recommendations


Formatting Recommendation : The recommendations are not visually appealing

In [39]:
def format_recommendations(recommendations):
    """Format recommendations for display."""
    formatted_output = []

    for search_term, articles in recommendations.items():
        formatted_output.append(f"\nSearch Term: **{search_term}**\n")
        if articles == "Result not found":
            formatted_output.append("Result not found")
        else:
            table_data = []
            for slug, title in articles:
                table_data.append([slug, title])

            formatted_output.append(tabulate(table_data, headers=["Slug", "Recommended Title"], tablefmt="grid"))

    return "\n".join(formatted_output)

Define Main Function:

In [40]:
def main():
    # Load and preprocess data
    content_df, labels_df, test_df = load_data()
    labels_df = preprocess_data(labels_df)

    # Generate sentence embeddings
    model, sentence_embeddings = fit_sentence_embeddings(labels_df)

    # Find the best threshold
    best_threshold, best_f1 = find_best_threshold(labels_df, model, sentence_embeddings)
    print(f"Best threshold: {best_threshold} with macro F1: {best_f1}")

    # Evaluate model performance
    evaluate_model(labels_df, model, sentence_embeddings, best_threshold)

    # Generate recommendations for the test set
    recommendations = recommend_articles_for_test(test_df, labels_df, model, sentence_embeddings, best_threshold, top_k=5)
    print(format_recommendations(recommendations))


In [41]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Best threshold: 0.4 with macro F1: 0.7350781998084903
              precision    recall  f1-score   support

NOT RELEVANT       0.58      0.65      0.62        43
    RELEVANT       0.87      0.84      0.85       123

    accuracy                           0.79       166
   macro avg       0.73      0.74      0.74       166
weighted avg       0.80      0.79      0.79       166


Search Term: **34 in. to 36 in. x 72 in. shower door**

+---------------------------------+---------------------------------+
| Slug                            | Recommended Title               |
+=================================+=================================+
| tips-for-selecting-shower-doors | Tips for Selecting Shower Doors |
+---------------------------------+---------------------------------+
| rug-sizes-for-your-space        | Rug Sizes for Your Space        |
+---------------------------------+---------------------------------+

Search Term: **outdoor prelit christmas tree**

+----------------------